# Função Gama


A Função Gama utilizada nesse programa tem como intuito receber os dados de posições e de doses de arquivos .csv, para tratá-los corretamente e aplicá-los em uma função matemática. 

Assim, seu retorno nos devolve os pontos como "aprovados" ou "reprovados", no sentido de verificar em quais pontos as doses são qualificadas, mediante comparação entre as calculadas e medidas.

In [2]:
import pandas as pd
import numpy as np

def Gamma_function(rc, rm, Dc, Dm):
    '''
    Função Gama que será utilizada ao longo do código,
    recebendo como parâmetros as medidas de posições (cm) 
    e doses (cGy) calculadas e medidas. 
    '''
    r_diff_x = abs(rc[0] - rm[0])
    r_diff_y = abs(rc[1] - rm[1])
    r_diff = np.sqrt(r_diff_x**2 + r_diff_y**2) # cálculo do módulo das posições
    D_diff = Dc - Dm # diferença entre doses
    
    F_g = np.sqrt((r_diff/0.3)**2 + (D_diff/0.03)**2) # cálculo da função gamma

    return F_g

def position_module(row):
    '''
    Função para calcular o módulo de posição, que chama a
    função Gamma já passando os parâmetros necessários.
    '''
    rm_x = row['Ponto Medido X (rm_x)']
    rm_y = row['Ponto Medido Y (rm_y)']
    rc_x = row['Ponto Calculado X (rc_x)']
    rc_y = row['Ponto Calculado Y (rc_y)']
    
    rm = (rm_x, rm_y) # coordenadas da posição Rm
    rc = (rc_x, rc_y) # coordenadas da posição Rc
    
    return Gamma_function(rc, rm, float(row['Dose Calculada (Dc)']), float(row['Dose Medida (Dm)']))

# Teste com novos dados

In [15]:
dados_planejados = "./Matrizes das Simulações para o Programa - Cópia de Simulações - Agosto.csv"

df_planned = pd.read_csv(dados_planejados)

df_planned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       175 non-null    float64
 1   Dose Medida (Dm)          875 non-null    float64
 2   Dose Calculada (Dc)       875 non-null    float64
 3   Ponto Medido X (rm_x)     875 non-null    float64
 4   Ponto Medido Y (rm_y)     875 non-null    float64
 5   Ponto Calculado X (rc_x)  875 non-null    float64
 6   Ponto Calculado Y (rc_y)  875 non-null    float64
dtypes: float64(7)
memory usage: 48.0 KB


## *Medidas de Agosto/2022*

* Abaixo se encontra a análise das 175 medidas realizadas em agosto/2022, onde previamente foram feitos os tratamentos dos dados, a utilização dos mesmos na função Gamma e a análise dos resultados.

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [42]:
# Leitura dos dados do mês de Agosto e criação do Dataframe
august_data = './Tabelas para o Programa - Programa - Agosto.csv'
df_augustData = pd.read_csv(august_data)

# Informações do Dataframe dos dados
print(df_augustData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Ponto Medido (rm)     175 non-null    object
 1   Ponto Calculado (rc)  175 non-null    object
 2   Dose Medida (Dm)      175 non-null    object
 3   Dose Calculada (Dc)   175 non-null    object
dtypes: object(4)
memory usage: 5.6+ KB
None


#### Tratamento dos dados

* Trata os dados do arquivo recebido padronizando a pontuação e dividindo as coordenadas X e Y em colunas diferentes.

In [43]:
# Dividir a coluna "Ponto Medido (rm)" em "Ponto Medido X (rm_x)" e "Ponto Medido Y (rm_y)"
df_augustData[['Ponto Medido X (rm_x)', 'Ponto Medido Y (rm_y)']] = df_augustData['Ponto Medido (rm)'].str.split(',', expand=True)

# Dividir a coluna "Ponto Calculado (rc)" em "Ponto Calculado X (rc_x)" e "Ponto Calculado Y (rc_y)"
df_augustData[['Ponto Calculado X (rc_x)', 'Ponto Calculado Y (rc_y)']] = df_augustData['Ponto Calculado (rc)'].str.split(',', expand=True)

# Converter as novas colunas para tipo numérico (float)
df_augustData[['Ponto Medido X (rm_x)', 'Ponto Medido Y (rm_y)', 'Ponto Calculado X (rc_x)', 'Ponto Calculado Y (rc_y)']] = df_augustData[['Ponto Medido X (rm_x)', 'Ponto Medido Y (rm_y)', 'Ponto Calculado X (rc_x)', 'Ponto Calculado Y (rc_y)']].astype(float)

# Excluir as colunas originais "Ponto Medido (rm)" e "Ponto Calculado (rc)" se desejar
df_augustData.drop(['Ponto Medido (rm)', 'Ponto Calculado (rc)'], axis=1, inplace=True)

# Substituir vírgulas por pontos nas colunas 'Dose Medida (Dm)' e 'Dose Calculada (Dc)'
df_augustData['Dose Medida (Dm)'] = df_augustData['Dose Medida (Dm)'].str.replace(',', '.')
df_augustData['Dose Calculada (Dc)'] = df_augustData['Dose Calculada (Dc)'].str.replace(',', '.')

# Converter as colunas de Dose Medida e Dose Calculada para tipos numéricos
df_augustData['Dose Medida (Dm)'] = pd.to_numeric(df_augustData['Dose Medida (Dm)'], errors='coerce')
df_augustData['Dose Calculada (Dc)'] = pd.to_numeric(df_augustData['Dose Calculada (Dc)'], errors='coerce')

#### Coluna Função Gamma

* Adiciona a coluna Função Gamma à tabela com seus valores calculados, com os valores arredondados na primeira casa decimal.

In [44]:
# Aplicar a função position_module a cada linha do DataFrame
df_augustData['Função Gama'] = df_augustData.apply(position_module, axis=1)

In [45]:
# Arredondar os valores da coluna 'Função Gama' para 1 casa decimal
df_augustData['Função Gama'] = df_augustData['Função Gama'].round(1)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente. Também adiciona uma nova coluna à tabela com o 'Status' dos pontos.

In [46]:
# Contar quantos valores na coluna 'Função Gama' são maiores que 1
maiores_que_um = (df_augustData['Função Gama'] > 1).sum()

# Contar quantos valores na coluna 'Função Gama' são menores que 1
menores_que_um = (df_augustData['Função Gama'] <= 1).sum()

# Total de valores na coluna 'Função Gama'
total_valores = len(df_augustData)

# Calcular a porcentagem de valores maiores que 1
porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

# Calcular a porcentagem de valores menores que 1
porcentagem_menores_que_um = (menores_que_um / total_valores) * 100

# Adicionar uma nova coluna com base nas condições
df_augustData['Status'] = np.where(df_augustData['Função Gama'] > 1, 'ponto reprovado', 'ponto aprovado')

In [47]:
# Imprimir os resultados
print("Valores referentes às medidas de Agosto")
print("Quantidade de valores aprovado, ou seja, menores ou iguais que 1:", menores_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", maiores_que_um)
print()
print(f"Porcentagem de valores maiores que 1: {porcentagem_maiores_que_um.round(2)}%")
print(f"Porcentagem de valores menores que 1: {porcentagem_menores_que_um.round(2)}%")

Valores referentes às medidas de Agosto
Quantidade de valores aprovado, ou seja, menores ou iguais que 1: 2
Quantidade de valores reprovados, ou seja, maiores que 1: 173

Porcentagem de valores maiores que 1: 98.86%
Porcentagem de valores menores que 1: 1.14%


#### Visualização dos dados tratados

* Visualiza a situação final da tabela, já com os dados tratados e novas colunas.

In [48]:
# Visualizar o DataFrame com os devidos tratamentos
print(df_augustData.head(5))

   Dose Medida (Dm)  Dose Calculada (Dc)  Ponto Medido X (rm_x)  \
0              50.3                 51.3                    1.8   
1              51.4                 51.7                    0.9   
2              51.5                 51.7                    0.0   
3              51.2                 51.7                   -0.9   
4              50.8                 51.3                   -1.8   

   Ponto Medido Y (rm_y)  Ponto Calculado X (rc_x)  Ponto Calculado Y (rc_y)  \
0                    1.8                       1.8                       1.8   
1                    1.8                       0.9                       1.8   
2                    1.8                       0.0                       1.8   
3                    1.8                      -0.9                       1.8   
4                    1.8                      -1.8                       1.8   

   Função Gama           Status  
0         33.3  ponto reprovado  
1         10.0  ponto reprovado  
2          6.7

## *Medidas de Fevereiro/2023*

* Abaixo se encontra a análise das 175 medidas realizadas em fevereiro/2023
, onde previamente foram feitos os tratamentos dos dados, a utilização dos mesmos na função Gamma e a análise dos resultados.

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [49]:
# Leitura dos dados do mês de Agosto e criação do Dataframe
february_data = './Tabelas para o Programa - Programa - Fevereiro.csv'
df_februaryData = pd.read_csv(february_data)

# Informações do Dataframe dos dados
print(df_februaryData.head())

  Ponto Medido (rm) Ponto Calculado (rc) Dose Medida (Dm) Dose Calculada (Dc)
0           1.8,1.8              1.8,1.8             50,7                51,3
1           0.9,1.8              0.9,1.8             51,6                51,7
2           0.0,1.8              0.0,1.8             51,3                51,7
3          -0.9,1.8             -0.9,1.8             51,2                51,7
4          -1.8,1.8             -1.8,1.8             50,8                51,3


#### Tratamento dos dados

* Trata os dados do arquivo recebido padronizando a pontuação e dividindo as coordenadas X e Y em colunas diferentes.

In [50]:
# Dividir a coluna "Ponto Medido (rm)" em "Ponto Medido X (rm_x)" e "Ponto Medido Y (rm_y)"
df_februaryData[['Ponto Medido X (rm_x)', 'Ponto Medido Y (rm_y)']] = df_februaryData['Ponto Medido (rm)'].str.split(',', expand=True)

# Dividir a coluna "Ponto Calculado (rc)" em "Ponto Calculado X (rc_x)" e "Ponto Calculado Y (rc_y)"
df_februaryData[['Ponto Calculado X (rc_x)', 'Ponto Calculado Y (rc_y)']] = df_februaryData['Ponto Calculado (rc)'].str.split(',', expand=True)

# Converter as novas colunas para tipo numérico (float)
df_februaryData[['Ponto Medido X (rm_x)', 'Ponto Medido Y (rm_y)', 'Ponto Calculado X (rc_x)', 'Ponto Calculado Y (rc_y)']] = df_februaryData[['Ponto Medido X (rm_x)', 'Ponto Medido Y (rm_y)', 'Ponto Calculado X (rc_x)', 'Ponto Calculado Y (rc_y)']].astype(float)

# Excluir as colunas originais "Ponto Medido (rm)" e "Ponto Calculado (rc)" se desejar
df_februaryData.drop(['Ponto Medido (rm)', 'Ponto Calculado (rc)'], axis=1, inplace=True)

# Substituir vírgulas por pontos nas colunas 'Dose Medida (Dm)' e 'Dose Calculada (Dc)'
df_februaryData['Dose Medida (Dm)'] = df_februaryData['Dose Medida (Dm)'].str.replace(',', '.')
df_februaryData['Dose Calculada (Dc)'] = df_februaryData['Dose Calculada (Dc)'].str.replace(',', '.')

# Converter as colunas de Dose Medida e Dose Calculada para tipos numéricos
df_februaryData['Dose Medida (Dm)'] = pd.to_numeric(df_februaryData['Dose Medida (Dm)'], errors='coerce')
df_februaryData['Dose Calculada (Dc)'] = pd.to_numeric(df_februaryData['Dose Calculada (Dc)'], errors='coerce')

#### Coluna Função Gamma

* Adiciona a coluna Função Gamma à tabela com seus valores calculados, com os valores arredondados na primeira casa decimal.

In [51]:
# Aplicar a função position_module a cada linha do DataFrame
df_februaryData['Função Gama'] = df_februaryData.apply(position_module, axis=1)

In [52]:
# Arredondar os valores da coluna 'Função Gama' para 1 casa decimal
df_februaryData['Função Gama'] = df_februaryData['Função Gama'].round(1)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente. Também adiciona uma nova coluna à tabela com o 'Status' dos pontos.

In [53]:
# Contar quantos valores na coluna 'Função Gama' são maiores que 1
maiores_que_um = (df_februaryData['Função Gama'] > 1).sum()

# Contar quantos valores na coluna 'Função Gama' são menores que 1
menores_que_um = (df_februaryData['Função Gama'] < 1).sum()

# Total de valores na coluna 'Função Gama'
total_valores = len(df_februaryData)

# Calcular a porcentagem de valores maiores que 1
porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

# Calcular a porcentagem de valores menores que 1
porcentagem_menores_que_um = (menores_que_um / total_valores) * 100

# Adicionar uma nova coluna com base nas condições
df_februaryData['Status'] = np.where(df_februaryData['Função Gama'] > 1, 'ponto reprovado', 'ponto aprovado')

In [54]:
# Imprimir os resultados
print("Valores referentes às medidas de Fevereiro")
print("Quantidade de valores aprovado, ou seja, menores que 1:", menores_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", maiores_que_um)
print()
print(f"Porcentagem de valores maiores que 1: {porcentagem_maiores_que_um.round(2)}%")
print(f"Porcentagem de valores menores que 1: {porcentagem_menores_que_um.round(2)}%")



Valores referentes às medidas de Fevereiro
Quantidade de valores aprovado, ou seja, menores que 1: 3
Quantidade de valores reprovados, ou seja, maiores que 1: 172

Porcentagem de valores maiores que 1: 98.29%
Porcentagem de valores menores que 1: 1.71%


#### Visualização dos dados tratados

* Visualiza a situação final da tabela, já com os dados tratados e novas colunas.

In [61]:
# Visualizar o DataFrame com os devidos tratamentos
print(df_februaryData.head())

   Dose Medida (Dm)  Dose Calculada (Dc)  Ponto Medido X (rm_x)  \
0              50.7                 51.3                    1.8   
1              51.6                 51.7                    0.9   
2              51.3                 51.7                    0.0   
3              51.2                 51.7                   -0.9   
4              50.8                 51.3                   -1.8   

   Ponto Medido Y (rm_y)  Ponto Calculado X (rc_x)  Ponto Calculado Y (rc_y)  \
0                    1.8                       1.8                       1.8   
1                    1.8                       0.9                       1.8   
2                    1.8                       0.0                       1.8   
3                    1.8                      -0.9                       1.8   
4                    1.8                      -1.8                       1.8   

   Função Gama           Status  
0         20.0  ponto reprovado  
1          3.3  ponto reprovado  
2         13.3

## *Salvando os Resultados*

* Abaixo, os resultados são salvos em um arquivo no formato Excel, com os resultados de cada mês em uma aba diferente

In [56]:
# Criando um arquivo 'resultados_analises_programa.xlsx' em formato Excel, que conterá duas abas, uma para cada DataFrame
with pd.ExcelWriter('resultados_analises_programa.xlsx', engine='xlsxwriter') as writer:
    # Salvando o primeiro DataFrame (Agosto)) em uma aba chamada 'Resultados - Agosto'
    df_augustData.to_excel(writer, sheet_name='Resultados - Agosto', index=False)

    # Salvando o segundo DataFrame (Fevereiro) em uma aba chamada 'Resultados - Fevereiro'
    df_februaryData.to_excel(writer, sheet_name='Resultados - Fevereiro', index=False)